### 层次化索引
层次化索引是pandas的一项重要功能，它能在一个轴上拥有多个（两个以上）索引级别
#### Series的分层索引

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.Series(np.random.randn(10), 
                 index=[['a','a','a','b','b','b','c','c','d','d'],[1,2,3,1,2,3,1,2,2,3]])

In [3]:
data

a  1   -0.430835
   2   -0.186451
   3   -1.840482
b  1   -0.464855
   2    0.472339
   3   -0.242715
c  1    0.206494
   2    1.792613
d  2    1.370484
   3    0.784530
dtype: float64

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

这样选取数据子集就很简单了

1.外层选取

In [5]:
data['b']

1   -0.464855
2    0.472339
3   -0.242715
dtype: float64

In [6]:
data['b':'c']

b  1   -0.464855
   2    0.472339
   3   -0.242715
c  1    0.206494
   2    1.792613
dtype: float64

In [7]:
data.loc[['b','d']]

b  1   -0.464855
   2    0.472339
   3   -0.242715
d  2    1.370484
   3    0.784530
dtype: float64

2.内层选取

In [10]:
data[:,2]

a   -0.186451
b    0.472339
c    1.792613
d    1.370484
dtype: float64

#### unstack/stack方法
该方法可以把两层分层索引的Series，重新安排到一个DataFrame中

In [12]:
data.unstack()

,1,2,3
a,-0.430835,-0.186451,-1.840482
b,-0.464855,0.472339,-0.242715
c,0.206494,1.792613,NaN
d,NaN,1.370484,0.784530


unstack逆运算stack

In [13]:
data.unstack().stack()

a  1   -0.430835
   2   -0.186451
   3   -1.840482
b  1   -0.464855
   2    0.472339
   3   -0.242715
c  1    0.206494
   2    1.792613
d  2    1.370484
   3    0.784530
dtype: float64

#### DataFrame的层次化索引
对于一个DataFrame,每条轴都可以有分层索引

In [15]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a','a','b','b'],[1,2,1,2]],
                    columns= [['zhao','hu','zhao'],['wang','li','zhang']])

In [16]:
frame

zhao  hu  zhao
    wang  li zhang
a 1    0   1     2
  2    3   4     5
b 1    6   7     8
  2    9  10    11

给各层起个名字吧

In [21]:
frame.index.names= ['key1','key2']

In [22]:
frame.columns.names = ['state','color']

In [23]:
frame

state     zhao  hu  zhao
color     wang  li zhang
key1 key2               
a    1       0   1     2
     2       3   4     5
b    1       6   7     8
     2       9  10    11

In [24]:
frame['zhao']

color      wang  zhang
key1 key2             
a    1        0      2
     2        3      5
b    1        6      8
     2        9     11

创建个MultiIndex复用

In [26]:
pd.MultiIndex.from_arrays([['zhao','zhao','hu'],['wang','li','zhang']],
                         names = ['state','color'])

MultiIndex(levels=[['hu', 'zhao'], ['li', 'wang', 'zhang']],
           labels=[[1, 1, 0], [1, 0, 2]],
           names=['state', 'color'])

#### 重排分级顺序
**swaplevel**交换各级别的顺序，它可以接收编号，也可以接收名称，最后返回一个新对象（数据不变）

In [27]:
frame.swaplevel('key1','key2')

,state,zhao,hu,zhao
,color,wang,li,zhang
key2,key1,,,
1,a,0,1,2
2,a,3,4,5
1,b,6,7,8
2,b,9,10,11


**sort_index**根据单个级别对数据进行排序

作用：默认根据行标签对所有行排序，或根据列标签对所有列排序，或根据指定某列或某几列对行排序。

调用方式：sort_index(axis=0, level=None, ascending=True, inplace=False, kind='quicksort', na_position='last', sort_remaining=True, by=None)

axis：0按照行名排序；1按照列名排序

level：默认None，否则按照给定的level顺序排列

ascending：默认True升序排列；False降序排列

inplace：默认False，否则排序之后的数据直接替换原来的数据框

kind：排序方法，{‘quicksort’, ‘mergesort’, ‘heapsort’}, default ‘quicksort’。似乎不用太关心。

na_position：缺失值默认排在最后{"first","last"}

by：按照某一列或几列数据进行排序，但是by参数貌似不建议使用

In [32]:
frame.sort_index(level='key2')

,state,zhao,hu,zhao
,color,wang,li,zhang
key1,key2,,,
a,1,0,1,2
b,1,6,7,8
a,2,3,4,5
b,2,9,10,11


In [39]:
frame.sort_index(level=0)

state     zhao  hu  zhao
color     wang  li zhang
key1 key2               
a    1       0   1     2
     2       3   4     5
b    1       6   7     8
     2       9  10    11

In [40]:
frame.swaplevel(0, 1).sort_index(level=0)

state     zhao  hu  zhao
color     wang  li zhang
key2 key1               
1    a       0   1     2
     b       6   7     8
2    a       3   4     5
     b       9  10    11

> 注：如果索引按照字典顺序从最外层开始排序，那么数据选择性能会更好--调用sort_index(level=0)或sort_index可以得到这样的结果

#### 根据级别汇总统计
DataFrame和Series的描述和汇总统计都有一个level选项，用它进行类似数据库中的groupby操作

In [41]:
frame.sum(level='key2')

state,zhao,hu,zhao
color,wang,li,zhang
key2,,,
1,6,8,10
2,12,14,16


In [42]:
frame.sum(level='color', axis = 1)

color      wang  li  zhang
key1 key2                 
a    1        0   1      2
     2        3   4      5
b    1        6   7      8
     2        9  10     11

#### 使用DataFrame的列进行索引
1.把某列变为行索引（set_index方法)

In [43]:
frame1 = pd.DataFrame({'a':range(7), 
                       'b':range(7,0,-1),
                       'c':['one','one','two','two','two','two','two'],
                       'd':[0,1,2,0,1,2,3]})

In [45]:
frame1

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,two,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [46]:
frame_setindex = frame1.set_index(['c','d'])

In [47]:
frame_setindex

a  b
c   d      
one 0  0  7
    1  1  6
two 2  2  5
    0  3  4
    1  4  3
    2  5  2
    3  6  1

默认变完后，那些列就移除了，当然也可以用**drop**选项保留下来

In [49]:
frame1.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
two 2  2  5  two  2
    0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

2.把行索引变为列（reset_index)

In [51]:
frame_setindex.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,two,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
